In [1]:
import requests
import time
from bs4 import BeautifulSoup
from random import sample 
import re
import glob as gb
import pandas as pd 
import shutil as sh
import os
from selenium import webdriver
import json
import csv
from datetime import datetime
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
import from crochet import setup


In [2]:
path = "C:\\Users\\padu\\Desktop\\Zillow\\"
os.chdir(path)

In [3]:
# zillow scraper class
class ZillowScraper(scrapy.Spider):
    # scraper/spider name
    name = 'zillow'
    
    # base URL
    base_url = 'https://www.zillow.com/charlotte-nc/rentals/2_p/?'
    
    # custom headers
    headers = {
         'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36'
    }
    
    # string query parameters
    params = {
       'searchQueryStae': '{"pagination":{"currentPage":2},"usersSearchTerm":"Charlotte,NC","mapBounds":{"west":-81.05677437207031,"east":-80.60496162792968,"south":34.99121373318643,"north":35.427128457340764},"regionSelection":[{"regionId":24043,"regionType":6}],"isMapVisible":true,"filterState":{},"isForSaleByOwner":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"isListVisible":true}'
    }
    
    # custom settings
    custom_settings = {
        # uncomment below settings to slow down the scraper
        'CONCURRENT_REQUESTS_PER_DOMAIN': 2,
        'DOWNLOAD_DELAY': 1
    }
    

In [4]:
  # crawler's entry point
def start_requests(self):
    # loop over page range 
    for page in range(0, 21,1):
        # parse params
        parsed_params = json.loads(self.params['searchQueryState'])

        # init next page
        parsed_params['pagination']['currentPage'] = str(page)

        # update string query parameters
        self.params['searchQueryState'] = json.dumps(parsed_params).replace(' ', '')

        # init next page URL
        next_page = self.base_url + urllib.parse.urlencode(self.params)

        # crawl next page URL
        yield scrapy.Request(url=next_page, headers=self.headers, callback=self.parse_links)

In [5]:
 # parse card links
def parse_links(self, res):
    # extract card links
    card_links = res.css('ul[class="photo-cards photo-cards_wow photo-cards_short"]')
    card_links = card_links.css('li')
    card_links = card_links.css('a.list-card-link::attr(href)')

    # loop over property cards
    for card in card_links:
        # crawl property listing page recursively
        yield res.follow(url=card.get(), headers=self.headers, callback=self.parse_listing)

In [6]:
           
# parse property listing
def parse_listing(self, res):
    
    # store listing HTML to local file
    with open('res.html', 'w') as f:
        f.write(res.text)


    
    # local listing HTML content
    content = ''

    # load local listing HTML file to extract data from it
    with open('res.html', 'r') as f:
        for line in f.read():
            content += line

    # init scrapy selector
    res = Selector(text=content)
    

    # extract feature list
    features = {
        'price': ''.join(res.css('div[class="ds-chip"]')
                              .css('h3[class="ds-price"] *::text')
                              .getall()),

        'address': ''.join(res.css('div[class="ds-chip"]')
                              .css('h1[class="ds-address-container"] *::text')
                              .getall()),

        'bedrooms': ' '.join(res.css('div[class="ds-chip"]')
                              .css('span[class="ds-bed-bath-living-area"] *::text')
                              .getall())
                              .replace('  ', '|')
                              .replace('| ', ':')
                              .split()[0]
                              .replace(':bd', ''),

        'bathrooms': ' '.join(res.css('div[class="ds-chip"]')
                  .css('span[class="ds-bed-bath-living-area"] *::text')
                  .getall())
                  .replace('  ', '|')
                  .replace('| ', ':')
                  .split()[1]
                  .replace(':ba', ''),

        'floor_area': ' '.join(res.css('div[class="ds-chip"]')
                  .css('span[class="ds-bed-bath-living-area"] *::text')
                  .getall())
                  .replace('  ', '|')
                  .replace('| ', ':')
                  .split()[2]
                  .replace(':', ' '),

        'zestimate': ''.join(res.css('div[class="ds-chip"]')
                                .css('div[class="ds-chip-removable-content"] *::text')
                                .getall())
                                .split('\u00ae:\u00a0')[-1],

        'description': res.css('div[class="Text-aiai24-0 sc-feJyhm erkQcD"]::text')
                          .get(),

        'agent_info': {

            'agent_name': res.css('ul[class="cf-listing-agent-info"] *::text')
                             .getall()[0],                              

            'agent_phone': res.css('ul[class="cf-listing-agent-info"] *::text')
                             .getall()[1]
        },

        'facts_and_features': {},

        'tax_history': {},

        'monthly_cost': {},

        'nearby_schools': [
                            ' '.join(Selector(text=school).css(' *::text').getall()) for school in
                            res.css('ul[class="ds-nearby-schools-list"]')
                               .css('li[class="sc-cMhqgX ikQQNx"]')
                               .getall()
                          ],

        'coordinates': {
            'latitude': '',
            'longitude': ''
        }

    }

    # try to extract facts and features
    try:
        facts = res.css('ul[class="ds-home-fact-list"]')
        facts = '|'.join(facts.css('li *::text').getall()).replace(':|', ':').split('|')

        # loop over facts
        for fact in facts:
            features['facts_and_features'][fact.split(':')[0]] = fact.split(':')[1]
    except:
        pass

    # try to extract tax history
    try:
        tax_history = ''.join([' '.join(Selector(text=ul).css('li *::text').getall()) for ul in res.css('ul[class="sc-dqBHgY kQzYMy"]').getall() if 'Tax assessed value:' in ul])

        # store tax history
        features['tax_history'] = {
            'Tax assessed value': tax_history.split('Tax assessed value:')[1].split()[0],
            'Annual tax amount': tax_history.split('Annual tax amount:')[1].split()[0]
        }

    except:
        pass

    # try to extract monthly cost
    try:
        monthly_cost = res.css('div[class="sc-1b8bq6y-6 kKSvPL"] *::text').getall()
        monthly_cost = '|'.join(monthly_cost).replace('|$', ':$').replace('Chevron Down', '')
        monthly_cost = [item.replace('|', ' ').strip().replace('Utilities ', 'Utilities:') for item in monthly_cost.split('||')]

        # loop over monthly cost table
        for item in monthly_cost:
            features['monthly_cost'][item.split(':')[0]] = item.split(':')[1]
    except:
        pass

    # try to extract coordinates
    script = [script for script in res.css('script[type="application/ld+json"]').getall() if 'latitude' in script][0]
    script = Selector(text=script).css('::text').get()
    script = json.loads(script)

    # store coordinates
    features['coordinates']['latitude'] = script['geo']['latitude']
    features['coordinates']['longitude'] = script['geo']['longitude']

    # store output to JSON file
    with open('zillow.jsonl', 'a') as f:
        f.write(json.dumps(features, indent=2) + '\n')

In [10]:
# main driver
if __name__ == '__main__':
    # run scraper
    process = CrawlerProcess()
    process.crawl(ZillowScraper)
    process.start()
    
#     debug data extraction logic
    ZillowScraper.parse_listing(ZillowScraper, '')
    
    
    

2021-03-05 12:54:43 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-05 12:54:43 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Windows-10-10.0.18362-SP0
2021-03-05 12:54:43 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-05 12:54:43 [scrapy.crawler] INFO: Overridden settings:
{'CONCURRENT_REQUESTS_PER_DOMAIN': 2, 'DOWNLOAD_DELAY': 1}
2021-03-05 12:54:43 [scrapy.extensions.telnet] INFO: Telnet Password: e664728075a903b5
2021-03-05 12:54:43 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-03-05 12:54:43 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy

ReactorNotRestartable: 

In [8]:
pwd

'C:\\Users\\padu\\Desktop\\Zillow'